<a href="https://colab.research.google.com/github/hongtaemin/-/blob/main/%EC%8B%9C%EC%95%BD_%EC%9B%B9%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip -q install playwright pandas openpyxl
!playwright install --with-deps chromium

import re
import asyncio
import pandas as pd
from datetime import datetime
from urllib.parse import urlencode
from playwright.async_api import async_playwright, TimeoutError as PWTimeout

HOME = "https://www.dslab.co.kr"
SEARCH_URL = "https://www.dslab.co.kr/shop/search.php"

def log(msg: str):
    print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}", flush=True)

def parse_price(text: str):
    if not text:
        return None
    nums = re.sub(r"[^\d]", "", text)
    return int(nums) if nums else None

async def js_set_value(page, css: str, value: str):
    # ✅ evaluate에는 인자를 1개만 넘길 수 있어서 dict로 묶어 전달
    return await page.evaluate(
        """(args) => {
            const el = document.querySelector(args.sel);
            if (!el) return false;
            el.value = args.val;
            el.dispatchEvent(new Event('input', {bubbles:true}));
            el.dispatchEvent(new Event('change', {bubbles:true}));
            return true;
        }""",
        {"sel": css, "val": value}
    )

async def js_click(page, css: str, label: str):
    try:
        ok = await page.evaluate(
            """(sel) => {
                const el = document.querySelector(sel);
                if (!el) return false;
                try { el.scrollIntoView({block:'center', inline:'center'}); } catch(e) {}
                el.click();
                return true;
            }""",
            css
        )
        if ok:
            log(f"  ✔ 클릭(JS): {label} ({css})")
        else:
            log(f"  ✖ 클릭(JS) 실패(요소없음): {label} ({css})")
        return ok
    except Exception as e:
        log(f"  ✖ 클릭(JS) 예외: {label} / {type(e).__name__}: {e}")
        return False

async def goto_search_direct(page, keyword: str, page_no: int = 1):
    params = {"q": keyword, "page": page_no, "ckattempt": 1}
    url = f"{SEARCH_URL}?{urlencode(params)}"
    log(f"  ↪ 백업: 검색 URL 직접 이동: {url}")
    await page.goto(url, wait_until="domcontentloaded")

async def ensure_on_search_page(page, keyword: str):
    """
    홈에서 검색어 입력 + 돋보기 클릭 후 search.php로 진입을 보장.
    실패하면 search.php?q=... 로 직접 이동(백업)
    """
    await page.goto(HOME, wait_until="domcontentloaded")
    await page.wait_for_selector("input#sch_str", timeout=20000)

    # 입력 (fill + JS value 세팅)
    await page.locator("input#sch_str").fill(keyword)
    await js_set_value(page, "input#sch_str", keyword)
    log(f"  ✔ 검색어 입력 완료: '{keyword}'")

    # 돋보기 클릭(JS)
    await js_click(page, "button#sch_submit", "돋보기")

    # URL 이동 감지 대기 → 실패 시 직접 이동
    try:
        await page.wait_for_url("**/shop/search.php**", timeout=12000)
        log(f"  ✔ search.php 이동 확인 / URL: {page.url}")
    except PWTimeout:
        log(f"  ⚠ URL 이동 미감지 / 현재 URL: {page.url}")
        await goto_search_direct(page, keyword, page_no=1)

    # 결과 영역 대기(#sct가 없어도 ul.item-list만 있으면 진행)
    try:
        await page.wait_for_selector("#sct, ul.item-list", timeout=15000)
        return True
    except PWTimeout:
        return False

async def get_page_items_count(page):
    return await page.locator("ul.item-list > li").count()

async def extract_price_from_nth_item(page, idx_in_page: int):
    """
    idx_in_page: 1부터 시작(nth-child)
    CSS → XPath(상대) → Abs XPath(최후) 순으로 가격 strong을 찾음
    """
    # 1) CSS
    css_price = f"ul.item-list > li:nth-child({idx_in_page}) .con-price .price-price strong"
    if await page.locator(css_price).count() > 0:
        txt = (await page.locator(css_price).first.inner_text(timeout=5000)).strip()
        price = parse_price(txt)
        if price is not None:
            return price, f"CSS / 원문:{txt}"

    # 2) XPath(상대)
    xp_price = f"//ul[contains(@class,'item-list')]/li[{idx_in_page}]//div[contains(@class,'con-price')]//strong"
    xp_sel = f"xpath={xp_price}"
    if await page.locator(xp_sel).count() > 0:
        txt = (await page.locator(xp_sel).first.inner_text(timeout=5000)).strip()
        price = parse_price(txt)
        if price is not None:
            return price, f"XPATH / 원문:{txt}"

    # 3) Abs XPath(최후)
    abs_xp = f"/html/body//ul[contains(@class,'item-list')]/li[{idx_in_page}]/a//div[contains(@class,'con-price')]//strong"
    abs_sel = f"xpath={abs_xp}"
    if await page.locator(abs_sel).count() > 0:
        txt = (await page.locator(abs_sel).first.inner_text(timeout=5000)).strip()
        price = parse_price(txt)
        if price is not None:
            return price, f"ABS_XPATH / 원문:{txt}"

    return None, "가격 요소를 찾지 못함(CSS/XPath/AbsXPath 실패)"

async def fetch_nth_price(page, keyword, n, excel_row):
    keyword = "" if keyword is None else str(keyword).strip()
    if len(keyword) < 2:
        return None, "FAIL:검색어 2글자 미만"

    try:
        n = int(n)
    except:
        return None, f"FAIL:순번 정수 변환 실패({n})"
    if n < 1:
        return None, f"FAIL:순번 1 미만({n})"

    log(f"{excel_row}행 ▶ 검색 시작: '{keyword}' / 전체 {n}번째")

    ok = await ensure_on_search_page(page, keyword)
    if not ok:
        return None, "FAIL:검색 페이지 진입 실패(#sct/ul.item-list 미검출)"

    # 1페이지 결과 로딩 대기
    try:
        await page.wait_for_selector("ul.item-list > li", timeout=20000)
    except PWTimeout:
        if await get_page_items_count(page) == 0:
            return None, "FAIL:검색 결과 0개"
        return None, "FAIL:검색결과 로딩 실패"

    page_size = await get_page_items_count(page)
    log(f"  ▶ 1페이지 아이템 수(page_size)={page_size}")
    if page_size == 0:
        return None, "FAIL:검색 결과 0개"

    # 전체 n번째 → 목표 페이지/인덱스 계산
    target_page = (n - 1) // page_size + 1
    idx_in_page = (n - 1) % page_size + 1
    log(f"  ▶ 목표: page={target_page}, page내순번={idx_in_page}")

    # 목표 페이지로 직접 이동(가장 안정적)
    if target_page != 1:
        await goto_search_direct(page, keyword, page_no=target_page)
        try:
            await page.wait_for_selector("ul.item-list > li", timeout=20000)
        except PWTimeout:
            return None, "FAIL:목표 페이지 로딩 실패"

    cnt = await get_page_items_count(page)
    log(f"  ▶ 목표 페이지 아이템 수={cnt}")
    if idx_in_page > cnt:
        return None, f"FAIL:{n}번째 없음(목표 페이지 {cnt}개)"

    price, detail = await extract_price_from_nth_item(page, idx_in_page)
    if price is None:
        return None, f"FAIL:{detail}"

    return price, f"OK:{price} / {detail}"

async def go_home(page):
    # 요구사항: 로고 클릭으로 홈 복귀 시도 → 실패하면 HOME 직접 이동
    try:
        a_kr = page.locator("a:has(span.logo-kr)")
        if await a_kr.count() > 0:
            try:
                await a_kr.first.click(force=True, timeout=5000)
                await page.wait_for_load_state("domcontentloaded")
                await page.wait_for_selector("input#sch_str", timeout=20000)
                return
            except:
                pass
    except:
        pass

    for sel in ["span.logo-kr", "span.logo-en"]:
        try:
            if await page.locator(sel).count() > 0:
                await js_click(page, sel, f"로고({sel})")
                await page.wait_for_load_state("domcontentloaded")
                await page.wait_for_selector("input#sch_str", timeout=20000)
                return
        except:
            pass

    await page.goto(HOME, wait_until="domcontentloaded")
    await page.wait_for_selector("input#sch_str", timeout=20000)

async def main_async(input_xlsx="input.xlsx", output_xlsx="output.xlsx"):
    log(f"엑셀 로드: {input_xlsx}")
    df = pd.read_excel(input_xlsx)
    log(f"엑셀 로드 완료: {len(df)}행 / {len(df.columns)}열")

    # 3열이 없으면 생성
    while df.shape[1] < 3:
        df[f"col{df.shape[1]+1}"] = None

    prices = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page(viewport={"width": 1280, "height": 900})

        # 검색어 검증 alert가 뜨면 자동 닫기
        page.on("dialog", lambda d: asyncio.create_task(d.accept()))

        for i, row in df.iterrows():
            excel_row = i + 1
            keyword = row.iloc[0]
            nth = row.iloc[1]

            log("=" * 80)
            log(f"{excel_row}행 시작 ▶ 검색명='{keyword}' / 순번='{nth}'")

            try:
                price, status = await fetch_nth_price(page, keyword, nth, excel_row)
            except Exception as e:
                price, status = None, f"EXCEPTION:{type(e).__name__}:{e}"

            prices.append(price)
            log(f"{excel_row}행 결과 ▶ 가격={price} / 상태={status}")

            await go_home(page)
            await asyncio.sleep(0.15)

        await browser.close()

    df.iloc[:, 2] = prices
    df.to_excel(output_xlsx, index=False)
    log(f"완료 ▶ {output_xlsx} 저장")

# Colab 실행
await main_async("input.xlsx", "output.xlsx")



Installing dependencies...
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entr

ERROR:asyncio:Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


[04:50:37] ================================================================================
[04:50:37] 2행 시작 ▶ 검색명='Sodium Silicate Solution  CP  규산나트륨용액 물유리' / 순번='1'
[04:50:37] 2행 ▶ 검색 시작: 'Sodium Silicate Solution  CP  규산나트륨용액 물유리' / 전체 1번째
[04:50:38]   ✔ 검색어 입력 완료: 'Sodium Silicate Solution  CP  규산나트륨용액 물유리'
[04:50:38]   ✔ 클릭(JS): 돋보기 (button#sch_submit)
[04:50:39]   ✔ search.php 이동 확인 / URL: https://www.dslab.co.kr/shop/search.php?q=Sodium+Silicate+Solution++CP++%EA%B7%9C%EC%82%B0%EB%82%98%ED%8A%B8%EB%A5%A8%EC%9A%A9%EC%95%A1+%EB%AC%BC%EC%9C%A0%EB%A6%AC
[04:50:40]   ▶ 1페이지 아이템 수(page_size)=3
[04:50:40]   ▶ 목표: page=1, page내순번=1
[04:50:40]   ▶ 목표 페이지 아이템 수=3
[04:50:40] 2행 결과 ▶ 가격=5830 / 상태=OK:5830 / CSS / 원문:5,830
[04:50:40] ================================================================================
[04:50:40] 3행 시작 ▶ 검색명='Dextrose Anhydrous  98.0  EP  덱스트로즈포도당' / 순번='1'
[04:50:40] 3행 ▶ 검색 시작: 'Dextrose Anhydrous  98.0  EP  덱스트로즈포도당' / 전체 1번째
[04:50:41]   ✔ 검색어 입력 완료: 'Dextrose